In [1]:
import tfim
import tfim_perturbation
import tfim_matrices
import tfim_rdm
import numpy as np
from scipy import sparse
from scipy.sparse import linalg as spla
from scipy import linalg
import matplotlib.pyplot as plt
from scipy import optimize
import progressbar
import itertools
import matplotlib.pyplot as pl
import schmidt_decomp

In [3]:
# Initial system specification
L = [3,3]
h_x_range = np.logspace(-2, -1, 80)
PBC = True
J = 1

In [27]:
# search for 4th order approximable Jij matrices
seed_limit = 100
max_order = 4

# Build lattice and basis
lattice = tfim.Lattice(L, PBC)
N = lattice.N
basis = tfim.IsingBasis(lattice)

# Specifying parameters needed
seed_range = range(seed_limit)

# Begin search
Jij_array = [];

for i in seed_range:
    Jij = tfim.Jij_instance(N,J,"bimodal",i) 
    Jij_array.append(Jij)

# Calculate energy array:
indices_array = []

for Jij in Jij_array:
    Energies = -tfim.JZZ_SK_ME(basis,Jij)
    GS_energy = np.min(Energies)
    GS_indices = np.nonzero(Energies == GS_energy)[0]
    indices_array.append(GS_indices)

# Search for Hamming distance 2
seed_list = []

for index, indices in enumerate(indices_array):
    if tfim_perturbation.judge(max_order, tfim_perturbation.Hamming_array(indices, basis), N):
        seed_list.append(index)

print(seed_list)


TypeError: Jij_instance() missing 1 required positional argument: 'even'

In [6]:
#construct random J matrix
# Jij = tfim.Jij_instance(N,J,"bimodal",Jij_seed) 
Jij_3 = np.array([[-1, 1, 1]])
Jij_4 = np.array([[0, 1, 0, -1],[1, 1, 1, 1]])

def bond_list(seed, N, PBC, xwidth, yheight):
    np.random.seed(seed)
    # Generates a random list of bonds with equal numbers of ferromagnetic and antiferromagnetic bonds
    if PBC == True:
        num_of_bonds = 2*N
    else:
        num_of_bonds = (xwidth - 1)*(yheight) + (xwidth)*(yheight - 1)
    if num_of_bonds%2 == 0:
        a1 = [-1 for i in range(num_of_bonds//2)]
    else:
        a1 = [-1 for i in range((num_of_bonds//2) + 1)]
    a2 = [1 for i in range(num_of_bonds//2)]
    a = list(np.random.permutation(a1+a2))
    return a

def make_Jij(N, b_list, lattice):
    #Goes through the list of bonds to make the jij matrix that tells you how all of the spins are bonded to each other
    
    bond_index = 0
    Jij = np.zeros((N,N))
    for i in range(0,N):
        NNs = lattice.NN(i)
        for j in NNs:
            if Jij[i][j] == 0:
                Jij[i][j] = b_list[bond_index]
                Jij[j][i] = b_list[bond_index]
                bond_index += 1
    return Jij

b_list = bond_list(Jij_seed, N, PBC, L[0], L[1])
Jij = make_Jij(N, b_list,lattice)

In [7]:
# List out all the spin_states, corresponding indices and energies
Energies = -tfim.JZZ_SK_ME(basis,Jij)
for index in range(2**N):
    print(index, basis.state(index), Energies[index])

0 [0 0 0 0 0 0 0 0 0] -6.0
1 [0 0 0 0 0 0 0 0 1] -4.0
2 [0 0 0 0 0 0 0 1 0] -8.0
3 [0 0 0 0 0 0 0 1 1] -6.0
4 [0 0 0 0 0 0 1 0 0] -0.0
5 [0 0 0 0 0 0 1 0 1] 2.0
6 [0 0 0 0 0 0 1 1 0] -2.0
7 [0 0 0 0 0 0 1 1 1] -0.0
8 [0 0 0 0 0 1 0 0 0] -2.0
9 [0 0 0 0 0 1 0 0 1] -4.0
10 [0 0 0 0 0 1 0 1 0] -0.0
11 [0 0 0 0 0 1 0 1 1] -2.0
12 [0 0 0 0 0 1 1 0 0] -0.0
13 [0 0 0 0 0 1 1 0 1] -2.0
14 [0 0 0 0 0 1 1 1 0] 2.0
15 [0 0 0 0 0 1 1 1 1] -0.0
16 [0 0 0 0 1 0 0 0 0] -6.0
17 [0 0 0 0 1 0 0 0 1] -4.0
18 [0 0 0 0 1 0 0 1 0] -8.0
19 [0 0 0 0 1 0 0 1 1] -6.0
20 [0 0 0 0 1 0 1 0 0] -0.0
21 [0 0 0 0 1 0 1 0 1] 2.0
22 [0 0 0 0 1 0 1 1 0] -2.0
23 [0 0 0 0 1 0 1 1 1] -0.0
24 [0 0 0 0 1 1 0 0 0] -2.0
25 [0 0 0 0 1 1 0 0 1] -4.0
26 [0 0 0 0 1 1 0 1 0] -0.0
27 [0 0 0 0 1 1 0 1 1] -2.0
28 [0 0 0 0 1 1 1 0 0] -0.0
29 [0 0 0 0 1 1 1 0 1] -2.0
30 [0 0 0 0 1 1 1 1 0] 2.0
31 [0 0 0 0 1 1 1 1 1] -0.0
32 [0 0 0 1 0 0 0 0 0] -4.0
33 [0 0 0 1 0 0 0 0 1] -2.0
34 [0 0 0 1 0 0 0 1 0] -6.0
35 [0 0 0 1 0 0 0 1 1] -4.0
36 [0 

In [8]:
# Build a list of excited state indices that are one Hamming distances away from ground states
GS_energy, GS_indices = tfim_perturbation.GS(Energies)

In [9]:
# Build exact matrix
V_exc = tfim_perturbation.V_exact(basis, lattice)

H_0_exc = tfim_perturbation.H_0_exact(Energies)

In [10]:
# Calculate approximated eigenvalues and eigenstates for range(h_x)
app_eigenvalues, app_eigenstates = tfim_perturbation.app_4_eigensystem_general_matrices(GS_indices, GS_energy, h_x_range, J, N, basis, Jij)

In [11]:
# Calculate exact eigenvalues and eigenstates for range(h_x)
exc_eigenvalues, exc_eigenstates = tfim_perturbation.exc_eigensystem(basis, h_x_range, lattice, Energies)

In [12]:
# Extract exact ground states
exc_GS_eigenstates = np.zeros((len(h_x_range), len(GS_indices), len(GS_indices)))

for i in range(len(h_x_range)):
    for m, j in enumerate(GS_indices):
        for n, k in enumerate(GS_indices):
            exc_GS_eigenstates[i, m, n] = exc_eigenstates[i, j, n]

In [13]:
# Extract exact ground energy

In [14]:
reordered_app_eigenstates = np.zeros([len(h_x_range), len(GS_indices), len(GS_indices)])
epsilon = 1*10**(-6)

for h_x_index in range(len(h_x_range)):
    if h_x_index < 2:
        reordered_app_eigenstates[h_x_index] = app_eigenstates[h_x_index]
    else: 
        for k in range(len(GS_indices)//2):
            fidelity_array = []
            for v1 in [reordered_app_eigenstates[h_x_index-1, :, 2*k], reordered_app_eigenstates[h_x_index-1, :, 2*k+1]]:
                for v2 in [app_eigenstates[h_x_index, :, 2*k], app_eigenstates[h_x_index, :, 2*k+1]]:
                    fidelity_array = np.append(fidelity_array,tfim_perturbation.fidelity(v1,v2))
            if abs(fidelity_array[0] - max(fidelity_array)) < epsilon:
                reordered_app_eigenstates[h_x_index, :, 2*k] = app_eigenstates[h_x_index, :, 2*k]
                reordered_app_eigenstates[h_x_index, :, 2*k + 1] = app_eigenstates[h_x_index, :, 2*k + 1]
            else:
                reordered_app_eigenstates[h_x_index, :, 2*k] = app_eigenstates[h_x_index, :, 2*k + 1]
                reordered_app_eigenstates[h_x_index, :, 2*k + 1] = app_eigenstates[h_x_index, :, 2*k]

In [15]:
reordered_exc_GS_eigenstates = np.zeros([len(h_x_range), len(GS_indices), len(GS_indices)])
epsilon = 1*10**(-12)

for h_x_index in range(len(h_x_range)):
    if h_x_index < 2:
        reordered_exc_GS_eigenstates[h_x_index] = exc_GS_eigenstates[h_x_index]
    else: 
        for k in range(len(GS_indices)//2):
            fidelity_array = []
            for v1 in [reordered_exc_GS_eigenstates[h_x_index-1, :, 2*k], reordered_exc_GS_eigenstates[h_x_index-1, :, 2*k+1]]:
                for v2 in [exc_GS_eigenstates[h_x_index, :, 2*k], exc_GS_eigenstates[h_x_index, :, 2*k+1]]:
                    fidelity_array = np.append(fidelity_array,tfim_perturbation.fidelity(v1,v2))
            if abs(fidelity_array[0] - max(fidelity_array)) < epsilon:
                reordered_exc_GS_eigenstates[h_x_index, :, 2*k] = exc_GS_eigenstates[h_x_index, :, 2*k]
                reordered_exc_GS_eigenstates[h_x_index, :, 2*k + 1] = exc_GS_eigenstates[h_x_index, :, 2*k + 1]
            else:
                reordered_exc_GS_eigenstates[h_x_index, :, 2*k] = exc_GS_eigenstates[h_x_index, :, 2*k + 1]
                reordered_exc_GS_eigenstates[h_x_index, :, 2*k + 1] = exc_GS_eigenstates[h_x_index, :, 2*k]

In [24]:
# Need to find the overall ground state wavefunction during perturbation
A = [1,3,5,7]
B = [0,2,4,6,8]
# this number 5 is arbitrary, only serves to give the index of the overall ground state
overall_GS = reordered_app_eigenstates[ :, :, np.argmin(app_eigenvalues.T[5])]
# overall_GS is a (perturbation length, eigenvector) size np array

In [22]:
A_basis, B_basis = schmidt_decomp.partition_basis(basis, GS_indices, A, B)

In [26]:
schmidt_decomp.entropy_analysis(basis, exc_eigenstates, exc_eigenvalues, GS_indices, A, B, overall_GS, h_x_range)

TypeError: 'numpy.int32' object is not iterable

In [ ]:
# For the paper, include 2 plots
# plot my perturbation theory result 2nd order y axis entropy and x axis h where since the quantum ground state
# degeneracy is fully split, then the entropy is just a constant since the eigenstate does not depend on h value, then compare it to the exact result
# where the ground state will depend on h, this plot will show that as h approaches 0
# 2nd plot is the error between the entropies of exact and approx. and we expect it to be a power law

# next step for the project: bound the error for the quantum ground state by performing perturbation theory on excited states

In [ ]:
# include a few instances of 3 by 3 and 4 by 4 instances and compare
# the exact vs the perturbation theory result for the paper